In [2]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import gensim as gensim

line = np.array('altijd november altijd regen altijd dit lege hart altijd'.split())
enc = OneHotEncoder()
data = enc.fit_transform(line.reshape(-1, 1))

corpus = {'altijd':1, 'dit':2, 'hart':3, 'lege':4, 'november':5, 'regen':6}
data = [corpus[k] for k in line]

model_BioSentVec = gensim.models.KeyedVectors.load_word2vec_format('/data/datasets/thema10/BioSentVec.vec.bin', binary=True)

#kijken of het model goed is geladen
len(model_BioSentVec.key_to_index) #16545452


16545452

In [3]:
a = model_BioSentVec['cancer']
b = model_BioSentVec['male']

print(type(a))
b.shape


<class 'numpy.ndarray'>


(200,)

In [15]:
def cosine_similarity(a,b):
    dot_product = np.dot(a,b)
    norm_vec1 = np.linalg.norm(a)
    norm_vec2 = np.linalg.norm(b)
    return dot_product / (norm_vec1 * norm_vec2)

print(cosine_similarity(a,b))

0.30956107


In [5]:
text = "Breast cancers with HER2 amplification have a higher risk of CNS metastasis and poorer prognosis."

In [17]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
import string

def preprocess_sentence(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.lower().split()
    
    text = [token for token in text if token not in stop_words]

    return text

sentence = preprocess_sentence(text)
print(sentence)


['breast', 'cancers', 'her2', 'amplification', 'higher', 'risk', 'cns', 'metastasis', 'poorer', 'prognosis']


[nltk_data] Downloading package stopwords to
[nltk_data]     /homes/blkoobs/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
def embed_sentence(sentence, model):
    """
    Embed een zin door de gemiddelde vector van de woorden te nemen.
    Woorden die niet in het model zitten, worden overgeslagen.
    """
    words = sentence.split()  # Simpele tokenisatie
    word_vectors = [model[word] for word in words if word in model]
    
    if not word_vectors:
        raise ValueError("Words not recognised by model")
    
    # Neem het gemiddelde van de woordvectoren
    embedding = np.mean(word_vectors, axis=0)
    return embedding

In [8]:
def compute_pca(X, n_components):
    X_centered = X - np.mean(X, axis=0)
    covariance_matrix = np.cov(X_centered, rowvar=False)
    eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)
    sorted_indices = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[sorted_indices]
    eigenvectors = eigenvectors[:, sorted_indices]
    top_eigenvectors = eigenvectors[:, :n_components]
    X_reduced = np.dot(X_centered, top_eigenvectors)

    return X_reduced

In [18]:
zin1 = "Breast cancers with HER2 amplification have a higher risk of CNS metastasis and poorer prognosis."
zin2 = "Breast cancers with HER2 amplification are more aggressive, have a higher risk of CNS metastasis, and poorer prognosis."
zin3 = "Furthermore, increased CREB expression in breast tumors is associated with poor prognosis, shorter survival and higher risk of metastasis."

for zin in [zin1,zin2,zin3]:
    X = embed_sentence(zin, model_BioSentVec)
    print(X.shape)
    #print(compute_pca(X, n_components=2))


(200,)
(200,)
(200,)
